In [18]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [92]:
class Vertice:
    
    def __init__(self, valor, direcionado=True):
        self.__dado = valor
        self.__direcionado = direcionado
        self.__arestas = set()
    
    def getDado(self):
        return self.__dado
    
    def setDado(self, valor):
        self.__dado = valor
        
    def getArestas(self):
        return self.__arestas
    
    def adicionarAresta(self, aresta):
        self.__arestas.add(aresta)
        
    def getArestasSaida(self):
        if self.__direcionado == False:
            return self.__arestas
        arestasDeSaida = []
        for aresta in self.__arestas:
            if aresta.getvOrigem() == self:
                arestasDeSaida.append(aresta)
        return arestasDeSaida
    
    def getArestasEntrada(self):
        if self.__direcionado == False:
            return self.__arestas
        arestasSaida = []
        for aresta in self.__arestas:
            if aresta.getvDestino() == self:
                arestasSaida.append(aresta)
        return arestasSaida
    
    def getGrau(self):
        return len(self.getArestasSaida())+ len(self.getArestasEntrada())

In [93]:
class Aresta:
    def __init__(self, vOrigem, vDestino, valor, direcionada=True):
        self.__vOrigem = vOrigem
        self.__vDestino = vDestino
        self.__valor = valor
        self.__direcionada = direcionada
        self.__vOrigem.adicionarAresta(self)
        self.__vDestino.adicionarAresta(self)
        
        
    def getvOrigem(self):
        return self.__vOrigem
    def getvDestino(self):
        return self.__vDestino
    def getValor(self):
        return self.__valor

In [94]:
class Grafo:
    def __init__(self, direcionado=True):
        self.__vertices = set()
        self.__arestas  = set()
        self.__direcionado = direcionado
        
    def setVertices(self, vertices):
        self.__vertices = vertices
        
    def setArestas(self, arestas):
        self.__arestas = arestas
        
    def getVertices(self):
        return self.__vertices
    
    def getArestas(self):
        return self.__arestas
    
    def checkHandShakingLemma(self):
        somaGraus = 0
        for v in self.getVertices():
            somaGraus+= v.getGrau()
        print(somaGraus)
        if somaGraus == len(self.getArestas())*2:
            return True
        else:
            return False

In [95]:
v1 = Vertice(1)
v2 = Vertice(2)
v3 = Vertice(3)
v4 = Vertice(4)
a1 = Aresta( v1, v2, 10, True )
a2 = Aresta( v2, v3, 20, True )
a3 = Aresta( v3, v4, 30, True )

In [96]:
G = Grafo()
G.setVertices({v1, v2, v3, v4})
G.setArestas({a1, a2, a3})

In [97]:
for v in G.getVertices():
    print(v.getDado())

2
4
1
3


In [98]:
for a in G.getArestas():
    print(a.getvOrigem().getDado(), end="")
    print(" ---> ", end="")
    print(a.getvDestino().getDado())

2 ---> 3
1 ---> 2
3 ---> 4


In [99]:
v1.getGrau()

1

In [100]:
G.checkHandShakingLemma()

6


True